In [ ]:
# imports

import os
import sys
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools

In [ ]:
# set path to files

# settings
datadir = '/eos/user/l/llambrec/dialstools-output'
eras = {
    '2024A': ['v1'],
    '2024B': ['v1'],
    '2024C': ['v1'],
    '2024D': ['v1'],
    '2024E': ['v1', 'v2'],
    '2024F': ['v1'],
    '2024G': ['v1'],
    '2024H': ['v1'],
    '2024I': ['v1', 'v2'],
    '2024J': ['v1'],
    '2025A': ['v1', 'v2'],
    '2025B': ['v1'],
    '2025C': ['v1', 'v2'],
    '2025D': ['v1'],
    '2025E': ['v1'],
    '2025F': ['v1', 'v2'],
    '2025G': ['v1']
}
dataset = 'ZeroBias'
reco = 'PromptReco'
mes = [
    'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1',
    'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2',
    'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3',
    'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_4'
]

# find files corresponding to settings
files = {}
nfiles = 0
for era, versions in eras.items():
    for version in versions:
        files[f'{era}-{version}'] = {}
        for me in mes:
            f = f'{dataset}-Run{era}-{reco}-{version}-DQMIO-{me}.parquet'
            f = os.path.join(datadir, f)
            files[f'{era}-{version}'][me] = f
            nfiles += 1

# existence check
missing = []
for era in files.keys():
    for me in files[era].keys():
        f = files[era][me]
        if not os.path.exists(f): missing.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')
else:
    print(f'Found {nfiles} files.')

In [ ]:
# print number of runs and lumisections

print('Finding number of runs and lumisections:')
for era in files.keys():
    for me in files[era].keys():
        f = files[era][me]
        df = iotools.read_parquet(f, columns=['run_number', 'ls_number'])
        run_numbers = dftools.get_runs(df, runcolumn='run_number')
        firstrun = run_numbers[0]
        lastrun = run_numbers[-1]
        nruns = len(run_numbers)
        nlumis = len(dftools.get_ls(df, lumicolumn='ls_number'))
        print(f'  - {era}: {nruns} runs, {nlumis} lumisections (run {firstrun} - {lastrun})')